# Count Documents Containing a Specific Ngram

This notebook allows you to count the number of documents in a project containing a specific word or phrase. You can also save document metadata to a dataframe, which you can explore in the notebook. This notebook also allows you to create a zip file including the documents containing this word or phrase as either json or txt files.

### Technical note

This notebook uses the NLTK package to build a custom tokenizer to tokenize project uni-, bi-, and trigrams. This tokenizer differs from the one used in the WE1S preprocessing pipeline. See the module's <a href="README.md" target="_blank">README.md</a> file for more information.

### INFO

__author__    = 'Lindsay Thomas'  
__copyright__ = 'copyright 2020, The WE1S Project'  
__license__   = 'MIT'  
__version__   = '2.0'  
__email__     = 'lindsaythomas@miami.edu'

## Settings

In [ ]:
# Python imports
import os
import csv
from pathlib import Path
from IPython.display import display, HTML

# Import scripts
%run scripts/count_tokens.py

# Define paths
current_dir     = %pwd
current_pathobj = Path(current_dir)
project_dir     = str(current_pathobj.parent.parent)
current_reldir  = current_dir.split("/write/")[1]
data_dir        = project_dir + '/project_data'
json_dir        = project_dir + '/project_data/json'
stopword_file   = '/home/jovyan/write/pub/templates/project_template/modules/topic_modeling/scripts/we1s_standard_stoplist.txt'

display(HTML('<p style="color: green;"><strong>Setup complete.</strong></p>'))

## 1. Configure Code

You must run all of the cells under this "Configure Code" section, even if you do not change the values.

### Set Content Field

In this cell, you will configure which field you would like to use as your input using the `content_field` variable. If your data includes full-text data in the `content` field, you can set `content_field = 'content'`. If you have data in your `bag_of_words` field, set `content_field = 'bag_of_words'`. If your data is in the `features` field, set `content_field = 'features'`. Follow this format exactly (including the quotation marks).

**Please note that if you use EITHER `bag_of_words` OR `features` as your text input field, you will only be able to count unigrams. See the module's <a href="README.md" target="_blank">README.md</a> file for more information on these fields.**

In [ ]:
## You may select 'content', 'features', or 'bag_of_words'
content_field = 'content'

if content_field not in ['content', 'features', 'bag_of_words']:
    msg = "The <code>content_field</code> variable must be 'content', 'features', or 'bag_of_words'."
    display(HTML('<p style="color: red;">' + msg + '</p>'))
else:
    msg = 'You have set the <code>content_field</code> variable to <code>' + content_field + '</code>.'
    display(HTML('<p style="color: green;">' + msg + '</p>'))

### Set Word or Phrase to Search For

In this cell you will configure the word or phrase you wish to search for. Set the `required_phrase` variable using the format `required_phrase = 'your word or phrase here'` (including the quotation marks).

You will choose whether to search using unigrams, bigrams, or trigrams in the following cell.

The code will return all instances of that word or phrase that occur across all project documents, regardless of whether they are upper- or lower-case. If the phrase you want to search for contains a stop word, like "first generation" (without the hyphen -- the stopword here is "first"), you should set `set_stopwords` below to `False`. You can view the standard list of WE1S stopwords used for all of our processes in the [project template folder](http://harbor.english.ucsb.edu:11111/edit/write/templates/multiple_topics_template/modules/02_MALLET/vectors/we1s_standard_stoplist.txt). To account for hyphenated phrases and other uses of punctuation, the cells below **will not remove hyphens**.

In [ ]:
# Set the word or phrase to search for
required_phrase = 'book'

if required_phrase == '':
    display(HTML('<p style="color: red;">The <code>required_phrase</code> is currently an empty string.</p>'))
else:
    msg = 'You have set the <code>required_phrase</code> to <code>' + required_phrase + '</code>.'
    display(HTML('<p style="color: green;">' + msg + '</p>'))

Configure the `set_length` variable below according to what you are searching for. The default is unigrams; to count bigrams or trigrams, comment out the unigram line, and uncomment the bi- or trigram line. 

**Note:** Because this code does not strip hyphens, hyphenated words like "first-generation" are considered unigrams.

In [ ]:
# Choose to analyze unigrams, bigrams, or trigrams
set_length = 'unigram'
# set_length = 'bigram'
# set_length = 'trigram'

if set_length not in ['unigram', 'bigram', 'trigram']:
    display(HTML("<p style=\"color: red;\">The <code>set_length</code> variable must be 'unigram', 'bigram', or 'trigram'.</p>"))
elif (set_length == 'bigram' or set_length == 'trigram' and content_field == 'bag_of_words') or (set_length == 'bigram' or set_length == 'trigram' and content_field == 'features'):
    msg = 'You cannot search for ' + set_length + 's with selected content field.'
    display(HTML('<p style="color: red;">' + msg + '</p>'))
else:
    msg = 'You have set the <code>set_length</code> variable to <code>' + set_length + '</code>.'
    display(HTML('<p style="color: green;">' + msg + '</p>'))

### Configure Punctuation Setting

This cell strips common punctuation from project documents. It will **NOT** strip hyphens, single or double, in order to account for hyphenated words and phrases such as "first-generation". Because this punctuation list is bespoke and not standardized (standardized options strip hyphens), some punctuation marks or other non-Unicode characters may make it through. You do not need to change anything about the below cell (unless you are interested in the frequency of punctuation marks, or @ signs, etc.), but you do need to run it. If you do not want to remove punctuation from your documents, you should set the `punctuations` variable to an empty string by uncommenting the line that says `punctuations = ''` in the cell below.

In [ ]:
# Define punctuation to strip
punctuations = "_______________________\'m\'n\'ve!()[]{};:\'\"\,<>./?@#$%^&*_~''``''"

# To strip no punctuation, uncomment the line below
# punctuations = ''

if punctuations == '':
    display(HTML('<p style="color: red;">You have elected not to strip any punctuation.</p>'))
else:
    msg = 'You have set the <code>punctuations</code> variable to <code>' + punctuations + '</code>.'
    display(HTML('<p style="color: green;">' + msg + '</p>'))   

### Configure Stop Word Setting

The default setting is to delete stop words before obtaining word frequencies using the WE1S standard stoplist. You can view this list in you project's `modules/topic_modeling/scripts` folder. You can edit this file for your project or create a custom stoplist. If you use a custom list, make sure that it is a plain text file with one word per line. Upload the file to your project and configure the `stopword_file` variable in the **Settings** cell to indicate the path to your custom stop word file.

If your data has already had the stop words you want removed or if you do not want to remove stop words, change the value of `set_stopwords` to `False`.

It is generally recommended to delete stop words from a document before obtaining bi- and/or trigram frequencies. This will result in "inexact" bi- and trigrams, however, as any stop words will be deleted *before* tokenization into bi- or trigrams. If you are interested in specific bi- or trigrams that contain stop words, such as "first" in "first generation" (without a hyphen), you may want to create a custom stop word list.

In [ ]:
# Delete stopwords from content fields before obtaining word frequencies.
# If set to True, stop words will be deleted. If set to false, stop words will not be deleted.
set_stopwords = True

if set_stopwords == True:
    display(HTML('<p style="color: green;">You have elected to strip stopwords.</p>'))
else:
    display(HTML('<p style="color: red;">You have elected not to strip stopwords.</p>'))

### Use Canonical Source Names (WE1S Data Only)

<p style="color: red;">Important: This cell should be run only if you are using WE1S project data.</p>

WE1S data contains a `source` field containing normalized titles of publication sources. If you wish to use these canonical source names to display publication titles in your output, change the `source_set` value to `True` (without quotation marks). Otherwise, the canonical source name will not be used.

In [ ]:
# Source_set feature.
# If set to False, the source field will not be used. 
# If set to True, the source field will be used.

source_set = True

msg = 'You have set the <code>source_set</code> variable to <code>' + str(source_set) + '</code>.'
display(HTML('<p style="color: green;">' + msg + '</p>'))

### Configure the CSV Filename

Run the below cell to set the name of the csv file you will save your results to. You do not need to change anything about the below cell unless you want to configure the file name differently.

In [ ]:
# The name of the csv file to save counts to
csv_file = 'counts_' + required_phrase + '.csv'

display(HTML('<p style="color:green;">You have set the csv filename to <code>' + csv_file + '</code>.</p>'))

## 2. Count Documents and Visualize Results

All cells after the below cell are optional in this section.

This cell creates a dataframe displaying metadata about each document in the project containing your search word or phrase and the number of times that word or phrase occurs in each document (in the "Count" column). The dataframe will print as output, sorted in descending order (from most to least) by count.

In [ ]:
# Overall_count implementation
file_list, df, bad_jsons = docs_by_search_term(json_dir, content_field, required_phrase, set_stopwords, source_set, set_length, 
                                         punctuations, stopword_file)
# Display the output
if df is not None:
    df_overall_count = df.sort_values('Count', ascending=False)
if df.empty:
    display(HTML('<p style="color: red;">No results found.</p>'))
else:
    display(HTML('<p>' + str(len(file_list)) + ' documents contain your search word or phrase.</p>'))
    if len(bad_jsons) > 0:
        msg = 'Warning! ' + str(len(bad_jsons)) + ' documents failed to load and will not be included in the counting. '
        msg += 'If this number is large, this will significantly affect your results.'
        display(HTML('<p style="color: red;">' + msg + '</p>'))
    display(HTML('<p style="color: green;">Dataframe created. View it by running the cell below.</p>'))

### View the Counts Dataframe 
The below cell uses a <a href="https://github.com/quantopian/qgrid" target="_blank">QGrid</a> widget to display count results in a dataframe. Click a column label to sort by that column. Click it again to reverse sort. Click the filter icon to the right of the column label to apply filters (for instance, reducing the table to only documents from specific sources). You can re-order the columns by dragging the column label.

In [ ]:
qgrid_widget = qgrid.show_grid(df_overall_count, grid_options=grid_options, show_toolbar=False)

qgrid_widget

### Save the Dataframe to a CSV File

The cell below will save the version of the dataframe you see displayed in the cell above. To save the full version of the dataframe (disregarding any filtering, etc you have done in the qgrid dataframe), skip the next cell, uncomment the code in the cell below it, and run that cell. 

Either cell will create a csv file in this module directory called "counts_YOUR-SEARCH-TERM-HERE.csv".

In [ ]:
# Save version of dataframe you see above to csv
changed_df = qgrid_widget.get_changed_df()

changed_df.to_csv(csv_file, index_label = 'Index')

display(HTML('<p style="color: green;">Dataframe saved to csv file named <code>' + csv_file + '</code></p>'))


In [ ]:
## Save original dataframe to csv, disregarding any changes you made in qgrid

# df_overall_count.to_csv(csv_file, index_label = 'Index')
# display(HTML('<p style="color: green;">Dataframe saved to csv file named <code>' + csv_file + '</code></p>'))

### Create Zip File of Project JSON Documents Containing Search Term

The cells below create a zip file titled "YOUR-SEARCH-TERM-HERE_json.zip" with copies of all json files in your project that contain your search word or phrase. The json files will include all document metadata and content data.

Run the below cell to set the name of the zip file you will create. You do not need to change anything about the below cell unless you want to configure the file name differently.

In [ ]:
# The path to the zip file you will create
zip_path = current_dir + '/' + required_phrase + '_json'

display(HTML('<p style="color:green;">You have set the zip filepath to <code>' + zip_path + '.zip</code>.</p>'))


Run the next cell to create the zip file.

In [ ]:
# Copy json files in file_list to separate folder and zip folder up for easy downloading.
zip_json(zip_path, file_list, json_dir)


### Create Zip File of Project Documents Containing Search Term as Text Files

The cells below create a zip file titled "YOUR-SEARCH-TERM-HERE_txt.zip" with text files containing data in the selected `content_field` from all json documents in your project that contain your search word or phrase.

The text files will have the same filenames as the original json documents, but they will not contain any of the documents' metadata.

Run the below cell to set the name of the zip file you will create and to confirm the `content_field` variable is set to the field you would like. You do not need to change anything about the below cell unless you want to configure the file name differently.

In [ ]:
# The path to the zip file you will create
txt_path = current_dir + '/' + required_phrase + '_txt'

display(HTML('<p style="color:green;">You have set the zip filepath to <code>' + txt_path + '.zip</code>.</p>'))
if content_field not in ['content', 'features', 'bag_of_words']:
    msg = "The <code>content_field</code> variable must be 'content', 'features', or 'bag_of_words'."
    display(HTML('<p style="color: red;">' + msg + '</p>'))
else:
    msg = 'You have set the <code>content_field</code> variable to <code>' + content_field + '</code>.'
    display(HTML('<p style="color: green;">' + msg + '</p>'))


If you would like to reconfigure the content field that will be copied to your text files, you can do so below by uncommenting the code and setting `content_field` to either `content`, `bag_of_words`, or `features`.

In [ ]:
# You may select 'content', 'features', or 'bag_of_words'
# content_field = 'content'

Run the next cell to create the zip file.

In [ ]:
# Copy content fields of json files in file_list to txt files in separate folder and zip folder up for easy downloading.
zip_txt(txt_path, file_list, json_dir, content_field)
